In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import json
import os
import datetime as dt
from tqdm import tqdm
import pandas as pd
import numpy as np


In [2]:
def f2cat(filename: str) -> str:
    return filename.split('.')[0]

class Simplified():
    def __init__(self, input_path='./input'):
        self.input_path = input_path

    def list_all_categories(self):
        files = ['apple.csv', 'banana.csv', 'circle.csv', 'pineapple.csv'] #os.listdir(os.path.join(self.input_path, 'train_simplified')) 
        print("files; ", files)
        return sorted([f2cat(f) for f in files], key=str.lower)

    def read_training_csv(self, category, nrows=None, usecols=None, drawing_transform=False):
        df = pd.read_csv(os.path.join(self.input_path, 'train_simplified', category + '.csv'),
                         nrows=nrows, parse_dates=['timestamp'], usecols=usecols)
        if drawing_transform:
            df['drawing'] = df['drawing'].apply(json.loads)
        return df
    def read_test_csv(self, category, skip_rows=30000, nrows=10000, usecols=None, drawing_transform=False):
        df=pd.read_csv(os.path.join(self.input_path, 'train_simplified', category + '.csv'),
                         parse_dates=['timestamp'], usecols=usecols)
        print("skip_rows")
        df=df.iloc[30000:40000]
        #print("len(df): ", len(df))
        if drawing_transform:
            df['drawing'] = df['drawing'].apply(json.loads)
        return df
    

In [3]:
s = Simplified('../input')
NCSVS = 4
categories = s.list_all_categories()
print(len(categories))


files;  ['apple.csv', 'banana.csv', 'circle.csv', 'pineapple.csv']
4


In [4]:
for y, cat in tqdm(enumerate(categories)):
    df = s.read_training_csv(cat, nrows=30000)
    #df
    df['y'] = y
    df['cv'] = (df.key_id // 10 ** 7) % NCSVS
    for k in range(NCSVS):
        filename = 'train_k{}.csv'.format(k)
        chunk = df[df.cv == k]
        chunk = chunk.drop(['key_id'], axis=1)
        if y == 0:
            chunk.to_csv(filename, index=False)
        else:
            chunk.to_csv(filename, mode='a', header=False, index=False)
#-------------------------- making test csv(s)
for y, cat in tqdm(enumerate(categories)):
    df = s.read_test_csv(cat, nrows=100, skip_rows=30000)
    print("df test, len(df): ", len(df))
    
    df['y'] = y
    df['cv'] = (df.key_id // 10 ** 7) % NCSVS
    for k in range(NCSVS):
        filename = 'test_k{}.csv'.format(k)
        chunk = df[df.cv == k]
        chunk = chunk.drop(['key_id'], axis=1)
        if y == 0:
            chunk.to_csv(filename, index=False)
        else:
            chunk.to_csv(filename, mode='a', header=False, index=False)
            
            

4it [00:02,  1.39it/s]
1it [00:01,  1.12s/it]

skip_rows
df test, len(df):  10000


2it [00:03,  1.35s/it]

skip_rows
df test, len(df):  10000


3it [00:03,  1.20s/it]

skip_rows
df test, len(df):  10000
skip_rows
df test, len(df):  10000


4it [00:05,  1.21s/it]


In [5]:
for k in tqdm(range(NCSVS)):
    filename = 'train_k{}.csv'.format(k)
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        df['rnd'] = np.random.rand(len(df))
        df = df.sort_values(by='rnd').drop('rnd', axis=1)
        df.to_csv(filename + '.gz', compression='gzip', index=False)
        os.remove(filename)
print(df.shape)

#-----------------------shuffling test csv(s)
for k in tqdm(range(NCSVS)):
    filename = 'test_k{}.csv'.format(k)
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        df['rnd'] = np.random.rand(len(df))
        df = df.sort_values(by='rnd').drop('rnd', axis=1)
        df.to_csv(filename + '.gz', compression='gzip', index=False)
        os.remove(filename)
print(df.shape)


  0%|          | 0/4 [00:00<?, ?it/s]

(30072, 7)


100%|██████████| 4/4 [00:11<00:00,  2.95s/it]

(9873, 7)


In [6]:
from IPython.display import FileLinks
FileLinks('.')


./
  train_k3.csv.gz
  __output__.json
  train_k0.csv.gz
  __notebook__.ipynb
  test_k0.csv.gz
  train_k2.csv.gz
  test_k2.csv.gz
  test_k1.csv.gz
  test_k3.csv.gz
  train_k1.csv.gz